# Quantum Kernel Machine Learning

To implement our use case, few things need to be considered. As usual, we will import our data, make some preprocessing such as check for missing values, split the data into training and testing datasets, rescale the data using normalization. An important step is feature extraction such as principal component analysis. We need to reduce our data from 43 features to 2 features. We could also use feature selection for dimension reduction. The encoding function is specified in a data_map function (for example data_map_12). The quantum kernels can then be used in a support vector classifier. Here, we will use scikit-learn (from sklearn.svm import SVC).

In [1]:
# Loading your IBM Quantum account: https://quantum-computing.ibm.com
# It requires us to sign with an IBMQ account.
from qiskit import IBMQ
IBMQ.save_account('Y O U R   A P I')

# Import utilities
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from functools import reduce

# sklearn imports
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Importing standard Qiskit libraries and Qiskit Machine Learning imports
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit.algorithms.optimizers import COBYLA

configrc.store_credentials:WARNING:2022-06-05 10:51:25,615: Credentials already present. Set overwrite=True to overwrite.


Backend objects can also be set up using the IBMQ package.
Assuming the credentials are already loaded onto your computer, you sign in with

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')

We can also see what additional backends are available. We will see simulators such as ibmq_qasm_simulator and real hardware such as ibmq_belem or ibmq_lima

In [3]:
for backend in provider.backends():
    print(backend)

ibmq_qasm_simulator
ibmq_armonk
ibmq_santiago
ibmq_bogota
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila


In [4]:
# seed for randomization, to keep outputs consistent
seed = 123456
algorithm_globals.random_seed = seed

In [5]:
# Encoding Functions
def data_map_8(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: np.pi*(m * n), x)
    return coeff

def data_map_9(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: (np.pi/2)*(m * n), 1 - x)
    return coeff

def data_map_10(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: np.pi*np.exp(((n - m)*(n - m))/8), x)
    return coeff

def data_map_11(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: (np.pi/3)*(m * n), 1/(np.cos(x)))
    return coeff

def data_map_12(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: np.pi*(m * n), np.cos(x))
    return coeff

In [18]:
# Quantum Feature Mapping with feature_dimension = 2 and reps = 2

qfm_default = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full')
print(qfm_default)
qfm_8 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_8)
print(qfm_8)
qfm_9 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_9)
print(qfm_9)
qfm_10 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_10)
print(qfm_10)
qfm_11 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_11)
print(qfm_11)
qfm_12 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_12)
print(qfm_12)
                                 


     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
 

In [7]:
# Use of a simulator (qasm_simulator)
qcomp_backend = QuantumInstance(BasicAer.get_backend('qasm_simulator'), shots=1024,
                                seed_simulator=seed, seed_transpiler=seed)
Q_Kernel_default = QuantumKernel(feature_map=qfm_default, quantum_instance=qcomp_backend)
Q_Kernel_8 = QuantumKernel(feature_map=qfm_8, quantum_instance=qcomp_backend)
Q_Kernel_9 = QuantumKernel(feature_map=qfm_9, quantum_instance=qcomp_backend)
Q_Kernel_10 = QuantumKernel(feature_map=qfm_10, quantum_instance=qcomp_backend)
Q_Kernel_11 = QuantumKernel(feature_map=qfm_11, quantum_instance=qcomp_backend)
Q_Kernel_12 = QuantumKernel(feature_map=qfm_12, quantum_instance=qcomp_backend)


In [8]:
# Alternativly we can use a real quantum computer (such as 'ibmq_bogota')

#real_qcomp_backend = QuantumInstance(provider.get_backend('ibmq_bogota'), shots=32)
#Q_Kernel_default = QuantumKernel(feature_map=qfm_default, quantum_instance=real_qcomp_backend)
#Q_Kernel_8 = QuantumKernel(feature_map=qfm_8, quantum_instance=real_qcomp_backend)
#Q_Kernel_9 = QuantumKernel(feature_map=qfm_9, quantum_instance=real_qcomp_backend)
#Q_Kernel_10 = QuantumKernel(feature_map=qfm_10, quantum_instance=real_qcomp_backend)
#Q_Kernel_11 = QuantumKernel(feature_map=qfm_11, quantum_instance=real_qcomp_backend)
#Q_Kernel_12 = QuantumKernel(feature_map=qfm_12, quantum_instance=real_qcomp_backend)

We process our data as we did for classical computing (load data, missing data, split of the data, normalization, PCA)

In [9]:
# Import dataset
data = '../data/datasets/neurons_test.csv'
df = pd.read_csv(data, delimiter=';')

# Drop row having at least 1 missing value
df = df.dropna()

# Creating an instance of Labelencoder
enc = LabelEncoder()
# Assigning numerical value and storing it
df[["Target"]] = df[["Target"]].apply(enc.fit_transform)

# Divide the data, y the variable to predict (Target) and X the features
X = df[df.columns[1:]]
y = df['Target']

# Splitting the data : training and test (20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Scaling the data
Normalize = preprocessing.StandardScaler()
# Transform data
X_train = Normalize.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = Normalize.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)

# Dimension Reduction with PCA (with two principal compoenents)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
# transform data
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)
# Define a new DataFrame with two column (the principal components)
component_columns = []
for x in (n+1 for n in range(2)):
    component_columns = component_columns + ['PCA_%i'%x]
X_train = pd.DataFrame(data = X_train, columns = component_columns)
X_test = pd.DataFrame(data = X_test, columns = component_columns)

print(X_train)
print(X_test)

        PCA_1      PCA_2
0    1.435834   4.081395
1    5.773557  10.022009
2   21.318734  -4.598521
3   -3.151619  -1.563259
4    0.668559   0.301271
5   -1.165522   0.369769
6   -0.958028  -1.190758
7   -3.716388  -1.479628
8   -3.968132  -1.303697
9    5.667760  -3.158447
10  -0.787751  -0.148257
11  -3.506257  -1.179403
12  -1.617898   0.087098
13  -2.480529  -0.935191
14  -4.335223  -1.390716
15  -1.048610  -0.730497
16   3.124327   3.135691
17   3.198947  -1.246605
18  -1.282505   2.405743
19  -4.256384  -1.492572
20  -1.835022   0.466993
21  -2.673096  -1.118469
22  -3.645921  -1.479659
23  -3.763482  -1.566059
24  -3.636642  -1.489347
25  -1.667317  -1.234876
26  -0.849866   5.925350
27   1.505479  -2.277992
28  -3.051316  -0.371275
29   6.733528   1.484747
30  -1.095088   3.026743
31   5.065872  -1.351583
      PCA_1     PCA_2
0 -3.569388 -0.568268
1  1.036985  6.469456
2 -3.763126 -0.032492
3  8.208539 -5.295320
4  1.561053 -1.076364
5 -4.648621 -3.235642
6  5.187216  3.940087

Then, we iterate other the different classifiers.

In [10]:
names = ["Q_Kernel_default", "Q_Kernel_8", "Q_Kernel_9",
         "Q_Kernel_10", "Q_Kernel_11", "Q_Kernel_12"]

classifiers = [
    SVC(kernel=Q_Kernel_default.evaluate),
    SVC(kernel=Q_Kernel_8.evaluate),
    SVC(kernel=Q_Kernel_9.evaluate),
    SVC(kernel=Q_Kernel_10.evaluate),
    SVC(kernel=Q_Kernel_11.evaluate),
    SVC(kernel=Q_Kernel_12.evaluate),
              ]


In [11]:
# iterate over classifiers and give classification test score
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print(f'Callable kernel classification test score for {name}: {score}')

Callable kernel classification test score for Q_Kernel_default: 0.125
Callable kernel classification test score for Q_Kernel_8: 0.125
Callable kernel classification test score for Q_Kernel_9: 0.125
Callable kernel classification test score for Q_Kernel_10: 0.125
Callable kernel classification test score for Q_Kernel_11: 0.25
Callable kernel classification test score for Q_Kernel_12: 0.25


As we also did previously in classical computing, we can provide metrics about our model (Accuracy, Precision, Recall, F1 score, cross validation or use the classification report).

In [13]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# Provide metrics over classifiers
for name, clf in zip(names, classifiers):
        print("\n")
        print(name)
        print("\n")
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
    
        print("\n")
        print("Print predicted data coming from X_test as new input data")
        print(y_pred)
        print("\n")
        print("Print real values\n")
        print(y_test)
        print("\n")
    
        print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
        print("Precision:", metrics.precision_score(y_test, y_pred, average='micro'))
        print("Recall:", metrics.recall_score(y_test, y_pred, average='micro'))
        print("f1 Score:", metrics.f1_score(y_test, y_pred, average='micro'))
        print("Cross Validation Mean:", cross_val_score(clf, X_train, y_train, cv=5).mean())
        print("Cross Validation Std:", cross_val_score(clf, X_train, y_train, cv=5).std())

print('Classification Report: \n')
print(classification_report(y_test,y_pred))



Q_Kernel_default




Print predicted data coming from X_test as new input data
[1 1 1 1 1 1 1 1]


Print real values

9     0
30    3
19    1
35    3
0     0
21    2
3     0
29    2
Name: Target, dtype: int64


Accuracy: 0.125
Precision: 0.125
Recall: 0.125
f1 Score: 0.125
Cross Validation Mean: 0.11904761904761904
Cross Validation Std: 0.10858813572372743


Q_Kernel_8




Print predicted data coming from X_test as new input data
[1 2 2 1 2 1 1 2]


Print real values

9     0
30    3
19    1
35    3
0     0
21    2
3     0
29    2
Name: Target, dtype: int64


Accuracy: 0.125
Precision: 0.125
Recall: 0.125
f1 Score: 0.125
Cross Validation Mean: 0.21904761904761902
Cross Validation Std: 0.07589227357385346


Q_Kernel_9




Print predicted data coming from X_test as new input data
[1 1 1 1 1 1 1 1]


Print real values

9     0
30    3
19    1
35    3
0     0
21    2
3     0
29    2
Name: Target, dtype: int64


Accuracy: 0.125
Precision: 0.125
Recall: 0.125
f1 Score: 0.125
Cross Validati

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## q_kernel_zz 

In [2]:
# Loading your IBM Quantum account(s)
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

# Import utilities
import numpy as np
import pandas as pd

# sklearn imports
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# Importing standard Qiskit libraries and Qiskit Machine Learning imports
from qiskit import Aer, QuantumCircuit, BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer

from typing import Union
from qiskit_machine_learning.exceptions import QiskitMachineLearningError

# seed for randomization, to keep outputs consistent
seed = 123456
algorithm_globals.random_seed = seed

# Define parameters
cv = 5 # Cross-validation 
feature_dimension = 5 # Features dimension
k_features = 5 # Feature selection
reps = 2 # Repetition
ibm_account = 'YOUR API'
quantum_backend = 'statevector_simulator'

# Import dataset
data = '../data/datasets/neurons_maha_soma.csv'
neuron = pd.read_csv(data, delimiter=',')

print(neuron)


df = neuron.head(22).copy()                    # Ganglion
df = pd.concat([df, neuron.iloc[320:340]])     # Granule
df = pd.concat([df, neuron.iloc[1493:1513]])   # Medium Spiny
df = pd.concat([df, neuron.iloc[1171:1191]])   # Parachromaffin
df = pd.concat([df, neuron.iloc[10031:10051]])   # Pyramidal

df = pd.concat([df, neuron.iloc[2705:2725]]) # Basket
df = pd.concat([df, neuron.iloc[22589:22609]]) # Bitufted
df = pd.concat([df, neuron.iloc[3175:3195]]) # Chandelier
df = pd.concat([df, neuron.iloc[22644:22664]]) # Double bouquet
df = pd.concat([df, neuron.iloc[3199:3219]]) # Martinotti
df = pd.concat([df, neuron.iloc[8260:8280]]) # Nitrergic

df = pd.concat([df, neuron.iloc[2255:2275]]) # Astrocytes
df = pd.concat([df, neuron.iloc[3306:3326]]) # Microglia


# Drop row having at least 1 missing value
df = df.dropna()

# Creating an instance of Labelencoder
enc = LabelEncoder()
# Assigning numerical value and storing it
df[["Target"]] = df[["Target"]].apply(enc.fit_transform)

from sklearn.model_selection import train_test_split
# We split our data to y (Target) and X (features)
y = df.loc[:, df.columns == 'Target']
# Features variables
X = df.loc[:, df.columns != ('Target')]
# Split data into train and test
# Option test_size = 0.2 means that we take 20% of the data for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
from sklearn.preprocessing import QuantileTransformer
Normalize = QuantileTransformer(n_quantiles=1000, output_distribution="uniform")
# Transform data
X_train = Normalize.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = Normalize.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)

from sklearn.tree import DecisionTreeClassifier

def embedded_decision_tree_classifier(X, y, k_features, output_folder=None):
    '''
    Here we use decision tree classifier to select features. We select the k best features (k_features)
    
    Inputs:
        - X (features) DataFrame
        - y (target) DataFrame
    
    '''
    
    print("\n")
    print("Decision Tree Regressor Features Importance: started")
    print("\n")

    # define the model
    model = DecisionTreeClassifier()
    # fit the model
    model.fit(X, y)
    # get importance
    importance = model.feature_importances_
    # Get features name
    feature_names = [f"{i}" for i in X.columns]

    # create a data frame to visualize features importance
    features_importance = pd.DataFrame({"Features": feature_names, "Importances":importance})
    features_importance.set_index('Importances')

    # Print features importance
    print("\n")
    print("Features Importances:")
    print("\n")
    print(features_importance)
    if output_folder is not None:
        features_importance.to_csv(output_folder+'Decision_Tree_Classifier_Features_Importance.csv', index=False)

    if output_folder is not None:
        # plot feature importance
        features_importance.plot(kind='bar',x='Features',y='Importances')
        pyplot.title('Decision Tree Classifier Features Importance')
        pyplot.tight_layout()
        pyplot.savefig(output_folder+'Decision_Tree_Classfier_Features_Importance.png')
    
    # Select the k most important features
    features_columns = []
    # Order the features importance dataframe
    df = pd.DataFrame(data = features_importance.sort_values(by='Importances', key=abs,ascending=False))
    # Put the k most important features in features_columns
    for x in range(k_features):
        features_columns = features_columns + [df.iloc[x][0]]

    # Create a new DataFrame with selected features
    df_data = pd.DataFrame(data = X, columns = features_columns)
    
    print("\n")
    print("Decision Tree Classifier Features Importance: DataFrame")
    print("\n")
    print(df_data)
    
    return df_data

X_train = embedded_decision_tree_classifier(X_train, y_train, k_features)
X_test = pd.DataFrame(data = X_test, columns = X_train.columns)

feature_dimension = X_train.shape[1] # Number of features
multiclass = None
output_folder = None
   
# We convert pandas DataFrame into numpy array
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()


# seed for randomization, to keep outputs consistent
seed = 123456
algorithm_globals.random_seed = seed

# Quantum Feature Mapping with feature_dimension = 5 and reps = 2
qfm_zz = ZZFeatureMap(feature_dimension=feature_dimension, reps=reps, entanglement="linear")
            
print(qfm_zz)
    
if 'ibmq_qasm_simulator' in quantum_backend:
    # Use of simulator
    # The use of these requires us to sign with an IBMQ account.
    # Assuming the credentials are already loaded onto your computer, you sign in with
    IBMQ.save_account(ibm_account, overwrite=True)
    IBMQ.load_account()
    provider = IBMQ.get_provider(hub='ibm-q')
    # What additional backends we have available.
    for backend in provider.backends():
        print(backend)
        
    sim = provider.backends.ibmq_qasm_simulator
    qcomp_backend = QuantumInstance(sim, shots=8192, seed_simulator=seed, seed_transpiler=seed)
    #qcomp_backend = QuantumInstance(BasicAer.get_backend(quantum_backend), shots=1024, seed_simulator=seed, seed_transpiler=seed)
    Q_Kernel_zz = QuantumKernel(feature_map=qfm_zz, quantum_instance=qcomp_backend)
                
else:
    if 'statevector_simulator' in quantum_backend:
        # Use of a simulator
        # The use of these requires us to sign with an IBMQ account.
        # Assuming the credentials are already loaded onto your computer, you sign in with
        IBMQ.save_account(ibm_account, overwrite=True)
        IBMQ.load_account()
        provider = IBMQ.get_provider(hub='ibm-q')
        # What additional backends we have available.
        for backend in provider.backends():
            print(backend)
        
        qcomp_backend = QuantumInstance(BasicAer.get_backend(quantum_backend), shots=1024, seed_simulator=seed, seed_transpiler=seed)
        Q_Kernel_zz = QuantumKernel(feature_map=qfm_zz, quantum_instance=qcomp_backend)
    else:
        if 'least_busy' in quantum_backend:
            # Use of least busy quantum hardware
            # The use of these requires us to sign with an IBMQ account.
            # Assuming the credentials are already loaded onto your computer, you sign in with
            IBMQ.save_account(ibm_account, overwrite=True)
            IBMQ.load_account()
            provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
            # What additional backends we have available.
            for backend in provider.backends():
                print(backend)
                
            device = least_busy(provider.backends(
                        filters=lambda x: x.configuration().n_qubits >= feature_dimension   # More than 5 qubits
                        and not x.configuration().simulator                                 # Not a simulator
                        and x.status().operational == True                                  # Operational backend
                    )
                )
            # Use of a real quantum computer
            print("Available device: ", device)
            quantum_backend = "%s"%device
                
            backend = provider.get_backend(quantum_backend)
            real_qcomp_backend = QuantumInstance(backend, shots=1024)
            Q_Kernel_zz = QuantumKernel(feature_map=qfm_zz, quantum_instance=real_qcomp_backend)

        else:
            # Use of a real quantum computer
            # The use of these requires us to sign with an IBMQ account.
            # Assuming the credentials are already loaded onto your computer, you sign in with
            IBMQ.save_account(ibm_account, overwrite=True)
            IBMQ.load_account()
            provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
            # What additional backends we have available.
            for backend in provider.backends():
                print(backend)
                    
            backend = provider.get_backend(quantum_backend)
            real_qcomp_backend = QuantumInstance(backend, shots=1024)
            Q_Kernel_zz = QuantumKernel(feature_map=qfm_zz, quantum_instance=real_qcomp_backend)
    
# QSVC model
model = QSVC(quantum_kernel=Q_Kernel_zz)
model.fit(X_train,y_train)
score = model.score(X_test, y_test)
print(f'Callable kernel classification test score for q_kernel_zz: {score}')
          
y_pred = model.predict(X_test)
        
print("\n")
print("Print predicted data coming from X_test as new input data")
print(y_pred)
print("\n")
print("Print real values\n")
print(y_test)
print("\n")
    
# K-Fold Cross Validation
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=cv)
score = np.zeros(cv)
i = 0
print(score)
for indices_train, indices_test in k_fold.split(X_train):
    #print(indices_train, indices_test)
    X_train_ = X_train[indices_train]
    X_test_ = X_train[indices_test]
    y_train_ = y_train[indices_train]
    y_test_ = y_train[indices_test]
 
    # fit classifier to data
    model.fit(X_train_, y_train_)

    # score classifier
    score[i] = model.score(X_test_, y_test_)
    i = i + 1

import math
print("cross validation scores: ", score)
cross_mean = sum(score) / len(score)
cross_var = sum(pow(x - cross_mean,2) for x in score) / len(score)  # variance
cross_std  = math.sqrt(cross_var)  # standard deviation
print("cross validation mean: ", cross_mean)
    
results = [metrics.accuracy_score(y_test, y_pred),metrics.precision_score(y_test, y_pred, average='micro'),metrics.recall_score(y_test, y_pred, average='micro'),metrics.f1_score(y_test, y_pred, average='micro'), cross_mean, cross_std]
    
metrics_dataframe = pd.DataFrame(results, index=["Accuracy", "Precision", "Recall", "F1 Score", "Cross-validation mean", "Cross-validation std"], columns=['q_kernel_zz'])

print('Classification Report: \n')
print(classification_report(y_test,y_pred))
        
print(metrics_dataframe)

               Target  Soma_Surface  N_stems  N_bifs  N_branch  N_tips  \
0            ganglion      1149.320      4.0   101.0     206.0   106.0   
1            ganglion      1511.830      3.0    70.0     143.0    74.0   
2            ganglion      1831.530      3.0    13.0      29.0    17.0   
3            ganglion      1291.270      6.0   109.0     224.0   116.0   
4            ganglion      3064.340      4.0    60.0     124.0    65.0   
...               ...           ...      ...     ...       ...     ...   
22686  double_bouquet       605.067      5.0   132.0     269.0   138.0   
22687  double_bouquet       920.949      6.0   121.0     248.0   128.0   
22688  double_bouquet       770.529      3.0   104.0     211.0   108.0   
22689  double_bouquet       478.078      4.0   158.0     320.0   163.0   
22690  double_bouquet       629.470      4.0    65.0     134.0    70.0   

        Width   Height   Depth     Type  ...  Bif_ampl_remote  Bif_tilt_local  \
0      249.09   493.80   33.63

/usr/local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (209). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (53). n_quantiles is set to n_samples.
  warnings.warn(
ibmqfactory.load_account:WARNING:2022-11-24 13:28:57,604: Credentials are already in use. The existing account in the session will be replaced.


ibmq_qasm_simulator
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila
ibm_nairobi
ibm_oslo
Callable kernel classification test score for q_kernel_zz: 0.4716981132075472


Print predicted data coming from X_test as new input data
[ 9 10  7  4 11  8  1  9 12  3 12  0  6 10  3  9 11  5  2 11 12 12  3 10
  7  7  5 11  7  8  6  3  0  7  6  1 11  9 10  5  0  8  3 11 12  3  9  3
  1 12  0  4  1]


Print real values

[ 9 10  7  4 11  2  4  9  5  1  6  0  8 10  7  9 11  5 10 11  2  3  5 10
  1  3 11 11 12  5  6  7  0  1  6  7 11  0 10  5  9  8  1 11  6  3  0  5
 12  2  9  4  1]


[0. 0. 0. 0. 0.]
cross validation scores:  [0.38095238 0.5        0.57142857 0.42857143 0.58536585]
cross validation mean:  0.4932636469221835
Classification Report: 

              precision    recall  f1-score   support

           0       0.50      0.50      0.50         4
           1       0.25      0.20      0.22         5
          

# Pegasos Quantum Support Vector Classifier: Binary Classification

There is also an alternative method to QSVC (which use the dual optimization from scikit-learn) using the Pegasos algorithm from Shalev-Shwartz where another SVM based algorithm benefits from the quantum kernel method. PegasosQSVC yields a training complexity that is independent of the size of the training set. This means that it could train faster than QSVC with large training sets. 

In [14]:
# Loading your IBM Quantum account(s)
from qiskit import IBMQ
IBMQ.save_account('YOUR API')

# Import utilities
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from functools import reduce

# sklearn imports
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Importing standard Qiskit libraries and Qiskit Machine Learning imports
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.algorithms import PegasosQSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit.algorithms.optimizers import COBYLA

# Backend objects can also be set up using the IBMQ package.
# The use of these requires us to sign with an IBMQ account.
# Assuming the credentials are already loaded onto your computer, you sign in with
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_lima')

# What additional backends we have available.
for backend in provider.backends():
    print(backend)
    
# seed for randomization, to keep outputs consistent
seed = 123456
algorithm_globals.random_seed = seed

# number of qubits is equal to the number of features
num_qubits = 2

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

# Encoding Functions

def data_map_8(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: np.pi*(m * n), x)
    return coeff

def data_map_9(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: (np.pi/2)*(m * n), 1 - x)
    return coeff

def data_map_10(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: np.pi*np.exp(((n - m)*(n - m))/8), x)
    return coeff

def data_map_11(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: (np.pi/3)*(m * n), 1/(np.cos(x)))
    return coeff

def data_map_12(x: np.ndarray) -> float:
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: np.pi*(m * n), np.cos(x))
    return coeff
    
# Quantum Feature Mapping with feature_dimension = 2 and reps = 2

qfm_default = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full')
print(qfm_default)
qfm_8 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_8)
print(qfm_8)
qfm_9 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_9)
print(qfm_9)
qfm_10 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_10)
print(qfm_10)
qfm_11 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_11)
print(qfm_11)
qfm_12 = PauliFeatureMap(feature_dimension=2,
                                    paulis = ['ZI','IZ','ZZ'],
                                 reps=2, entanglement='full', data_map_func=data_map_12)
print(qfm_12)
                                 
print(qfm_8.draw())

# Use of a simulator (qasm_simulator)

qcomp_backend = QuantumInstance(BasicAer.get_backend('qasm_simulator'), shots=1024,
                                seed_simulator=seed, seed_transpiler=seed)
Q_Kernel_default = QuantumKernel(feature_map=qfm_default, quantum_instance=qcomp_backend)
Q_Kernel_8 = QuantumKernel(feature_map=qfm_8, quantum_instance=qcomp_backend)
Q_Kernel_9 = QuantumKernel(feature_map=qfm_9, quantum_instance=qcomp_backend)
Q_Kernel_10 = QuantumKernel(feature_map=qfm_10, quantum_instance=qcomp_backend)
Q_Kernel_11 = QuantumKernel(feature_map=qfm_11, quantum_instance=qcomp_backend)
Q_Kernel_12 = QuantumKernel(feature_map=qfm_12, quantum_instance=qcomp_backend)

names = ["Q_Kernel_default", "Q_Kernel_8", "Q_Kernel_9",
         "Q_Kernel_10", "Q_Kernel_11", "Q_Kernel_12"]

classifiers = [
    PegasosQSVC(quantum_kernel=Q_Kernel_default, C=C, num_steps=tau),
    PegasosQSVC(quantum_kernel=Q_Kernel_8, C=C, num_steps=tau),
    PegasosQSVC(quantum_kernel=Q_Kernel_9, C=C, num_steps=tau),
    PegasosQSVC(quantum_kernel=Q_Kernel_10, C=C, num_steps=tau),
    PegasosQSVC(quantum_kernel=Q_Kernel_11, C=C, num_steps=tau),
    PegasosQSVC(quantum_kernel=Q_Kernel_12, C=C, num_steps=tau),
              ]

# Import dataset
data = '../data/datasets/neurons_test.csv'
df = pd.read_csv(data, delimiter=';')

# Drop row having at least 1 missing value
df = df.dropna()

# Creating an instance of Labelencoder
enc = LabelEncoder()
# Assigning numerical value and storing it
df[["Target"]] = df[["Target"]].apply(enc.fit_transform)

# Divide the data, y the variable to predict (Target) and X the features
X = df[df.columns[1:]]
y = df['Target']

# Splitting the data : training and test (20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Scaling the data
Normalize = preprocessing.StandardScaler()
# Transform data
X_train = Normalize.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = Normalize.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)

# Dimension Reduction with PCA (with two principal compoenents)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
# transform data
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)
# Define a new DataFrame with two column (the principal components)
component_columns = []
for x in (n+1 for n in range(2)):
    component_columns = component_columns + ['PCA_%i'%x]
X_train = pd.DataFrame(data = X_train, columns = component_columns)
X_test = pd.DataFrame(data = X_test, columns = component_columns)

print(X_train)
print(X_test)

# iterate over classifiers
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print(f'Callable kernel classification test score for {name}: {score}')
        
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# Provide metrics over classifiers
for name, clf in zip(names, classifiers):
        print("\n")
        print(name)
        print("\n")
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
    
        print("\n")
        print("Print predicted data coming from X_test as new input data")
        print(y_pred)
        print("\n")
        print("Print real values\n")
        print(y_test)
        print("\n")
    
        print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
        print("Precision:", metrics.precision_score(y_test, y_pred, average='micro'))
        print("Recall:", metrics.recall_score(y_test, y_pred, average='micro'))
        print("f1 Score:", metrics.f1_score(y_test, y_pred, average='micro'))
        print("Cross Validation Mean:", cross_val_score(clf, X_train, y_train, cv=5).mean())
        print("Cross Validation Std:", cross_val_score(clf, X_train, y_train, cv=5).std())
        
        
        results = [metrics.accuracy_score(y_test, y_pred),metrics.precision_score(y_test, y_pred, average='micro'),metrics.recall_score(y_test, y_pred, average='micro'),metrics.f1_score(y_test, y_pred, average='micro'), cross_val_score(clf, X_train, y_train, cv=5).mean(), cross_val_score(clf, X_train, y_train, cv=5).std()]
        metrics_dataframe = pd.DataFrame(results, index=["Accuracy", "Precision", "Recall", "F1 Score", "Cross-validation mean", "Cross-validation std"], columns=[name])

        print('Classification Report: \n')
        print(classification_report(y_test,y_pred))
    
        print('Metrics:')
        print(metrics_dataframe)
    
    


configrc.store_credentials:WARNING:2022-06-05 11:39:04,764: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2022-06-05 11:39:05,848: Credentials are already in use. The existing account in the session will be replaced.


ibmq_qasm_simulator
ibmq_armonk
ibmq_santiago
ibmq_bogota
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                            ├
     │  PauliFeatureMap(x[0],x[1]) │
q_1: ┤1                            ├
     └─────────────────────────────┘
     ┌─────────────────────────────┐
q_0: ┤0                           

ValueError: Only binary classification is supported

## Quantum Kernel Training

It is also possible to train a quantum kernel with Quantum Kernel Alignment (QKA) that iteratively adapts a parametrized quantum kernel to a dataset and converging to the maximum SVM margin at the same time. To implement it, we prepare the dataset as usual and define the quantum feature map. Then, we will use QuantumKernelTrained.fit method to train the kernel parameters and pass it to a machine learning model.
Source: https://lab.quantum-computing.ibm.com/user/5ae8692a0f0205003930696d/lab/workspaces/auto-s/tree/qiskit-tutorials/qiskit-machine-learning/08_quantum_kernel_trainer.ipynb

In [10]:
# Loading your IBM Quantum account(s)
from qiskit import IBMQ

# Import utilities
import numpy as np
import pandas as pd

# sklearn imports
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

# Importing standard Qiskit libraries and Qiskit Machine Learning imports
from qiskit import BasicAer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.providers.aer import AerSimulator
from qiskit.visualization import circuit_drawer
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC

# Import dataset
data = '../data/datasets/neurons.csv'
neuron = pd.read_csv(data, delimiter=';')

# Select a subset of the data composed of three classes
df = neuron.head(20).copy()                    # Ganglion
df = pd.concat([df, neuron.iloc[373:393]])     # Granule
df = pd.concat([df, neuron.iloc[17033:17053]]) # Basket

# seed for randomization, to keep outputs consistent
seed = 123456
algorithm_globals.random_seed = seed

# Drop row having at least 1 missing value
df = df.dropna()

# Creating an instance of Labelencoder
enc = LabelEncoder()
# Assigning numerical value and storing it
df[["Target"]] = df[["Target"]].apply(enc.fit_transform)

# Divide the data, y the variable to predict (Target) and X the features
X = df[df.columns[1:]]
y = df['Target']

# Splitting the data : training and test (20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Scaling the data
Normalize = preprocessing.StandardScaler()
# Transform data
X_train = Normalize.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = Normalize.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)

# Dimension Reduction with PCA (with two principal compoenents)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
# transform data
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)
# Define a new DataFrame with two column (the principal components)
component_columns = []
for x in (n+1 for n in range(2)):
    component_columns = component_columns + ['PCA_%i'%x]
X_train = pd.DataFrame(data = X_train, columns = component_columns)
X_test = pd.DataFrame(data = X_test, columns = component_columns)

# Define some parameters
feature_dimension = 2 # number of features
reps = 2 # number of repetitions
quantum_backend = 'qasm_simulator' # quantum backend
cv = 2 # Cross-validation
circuits = 2 # number of circuits

# The use of these requires us to sign with an IBMQ account.
# Assuming the credentials are already loaded onto your computer, you sign in with
IBMQ.save_account('YOUR API')
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')

# Define a callback class for our optimizer
class QKTCallback:
    """Callback wrapper class."""

    def __init__(self) -> None:
        self._data = [[] for i in range(5)]

    def callback(self, x0, x1=None, x2=None, x3=None, x4=None):
        """
        Args:
            x0: number of function evaluations
            x1: the parameters
            x2: the function value
            x3: the stepsize
            x4: whether the step was accepted
        """
        self._data[0].append(x0)
        self._data[1].append(x1)
        self._data[2].append(x2)
        self._data[3].append(x3)
        self._data[4].append(x4)

    def get_callback_data(self):
        return self._data

    def clear_callback_data(self):
        self._data = [[] for i in range(5)]
    
# seed for randomization, to keep outputs consistent
seed = 123456
algorithm_globals.random_seed = seed

# Rotational layer to train. We rotate each qubit the same amount.
from qiskit.circuit import ParameterVector
user_params = ParameterVector("θ", 1)
fm0 = QuantumCircuit(circuits) # Number of circuits
fm0.ry(user_params[0], 0)
fm0.ry(user_params[0], 1)

# Use ZZFeatureMap 
qfm_zz = ZZFeatureMap(feature_dimension)

# Create the feature map
fm = fm0.compose(qfm_zz)

print(circuit_drawer(fm))
print(f"Trainable parameters: {user_params}")

print(qfm_zz)

if 'qasm_simulator' in quantum_backend:
    # Use of a simulator
    qcomp_backend = QuantumInstance(BasicAer.get_backend(quantum_backend), shots=1024, seed_simulator=seed, seed_transpiler=seed)
    Q_Kernel_zz = QuantumKernel(feature_map=fm, user_parameters=user_params, quantum_instance=qcomp_backend)
else:
    # Use of a real quantum computer
    real_qcomp_backend = QuantumInstance(provider.get_backend(quantum_backend), shots=1024)
    Q_Kernel_zz = QuantumKernel(feature_map=fm, user_parameters=user_params, quantum_instance=real_qcomp_backend)


# Instantiate quantum kernel
quant_kernel = QuantumKernel(fm, user_parameters=user_params, quantum_instance=qcomp_backend)

# Set up the optimizer
cb_qkt = QKTCallback()
spsa_opt = SPSA(maxiter=10, callback=cb_qkt.callback, learning_rate=0.05, perturbation=0.05)

# Instantiate a quantum kernel trainer
qkt = QuantumKernelTrainer(
    quantum_kernel=quant_kernel, loss="svc_loss", optimizer=spsa_opt, initial_point=[np.pi / 2]
)

# Train the kernel using QKT directly
qka_results = qkt.fit(X_train, y_train)
optimized_kernel = qka_results.quantum_kernel
print(qka_results)

# Use QSVC for classification
qsvc = QSVC(quantum_kernel=optimized_kernel)

# Fit the QSVC
qsvc.fit(X_train, y_train)

# Predict the labels
y_pred = qsvc.predict(X_test)

# Evalaute the test accuracy
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=y_pred)
print(f"accuracy test: {accuracy_test}")

# Print predicted values and real values of the X_test dataset
print("\n")
print("Print predicted data coming from X_test as new input data")
print(y_pred)
print("\n")
print("Print real values\n")
print(y_test)
print("\n")
    
# Print accuracy metrics of the model
results = [metrics.accuracy_score(y_test, y_pred),metrics.precision_score(y_test, y_pred, average='micro'),metrics.recall_score(y_test, y_pred, average='micro'),metrics.f1_score(y_test, y_pred, average='micro'), cross_val_score(qsvc, X_train, y_train, cv=cv).mean(), cross_val_score(qsvc, X_train, y_train, cv=cv).std()]
metrics_dataframe = pd.DataFrame(results, index=["Accuracy", "Precision", "Recall", "F1 Score", "Cross-validation mean", "Cross-validation std"], columns=['q_kernel_training'])
print('Classification Report: \n')
print(classification_report(y_test,y_pred))
        
metrics_dataframe


configrc.store_credentials:WARNING:2022-07-08 18:52:43,605: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2022-07-08 18:52:45,795: Credentials are already in use. The existing account in the session will be replaced.


     ┌──────────┐┌──────────────────────────┐
q_0: ┤ Ry(θ[0]) ├┤0                         ├
     ├──────────┤│  ZZFeatureMap(x[0],x[1]) │
q_1: ┤ Ry(θ[0]) ├┤1                         ├
     └──────────┘└──────────────────────────┘
Trainable parameters: θ, ['θ[0]']
     ┌──────────────────────────┐
q_0: ┤0                         ├
     │  ZZFeatureMap(x[0],x[1]) │
q_1: ┤1                         ├
     └──────────────────────────┘
{   'optimal_parameters': {ParameterVectorElement(θ[0]): 1.3059540613374132},
    'optimal_point': array([1.30595406]),
    'optimal_value': 18.27409165606242,
    'optimizer_evals': 30,
    'optimizer_time': None,
    'quantum_kernel': <qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x12e514f40>}
accuracy test: 0.5595238095238095


Print predicted data coming from X_test as new input data
[2 2 2 1 0 0 1 1 0 1 1 2]


Print real values

377      2
17034    0
381      2
17051    0
17052    0
374      2
9        1
17041    0
388      2
378

,q_kernel_zz
Accuracy,0.416667
Precision,0.416667
Recall,0.416667
F1 Score,0.416667
Cross-validation mean,0.416667
Cross-validation std,0.041667


# Quantum Kernel Alignment with Qiskit Runtime

Source: https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials/qka.html

In [9]:
import sys

sys.path.insert(0, "..")  # Add program_source directory to the path

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_montreal")

# Loading your IBM Quantum account(s)
from qiskit import IBMQ
IBMQ.save_account('YOUR API')
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')

# Import utilities
import numpy as np
import pandas as pd

# Import dataset
data = '../data/datasets/neurons.csv'
neuron = pd.read_csv(data, delimiter=';')

df = neuron.head(371).copy()                     # Ganglion
df = pd.concat([df, neuron.iloc[373:1410]])    # Granule
df = pd.concat([df, neuron.iloc[1411:2272]])   # Medium Spiny
#df = pd.concat([df, neuron.iloc[2273:2797]])   # Parachromaffin
#df = pd.concat([df, neuron.iloc[2840:3294]])   # Purkinje
#df = pd.concat([df, neuron.iloc[3295:17032]])  # Pyramidal

#df = pd.concat([df, neuron.iloc[17033:17505]]) # Basket
#df = pd.concat([df, neuron.iloc[17506:17572]]) # Bitufted
#df = pd.concat([df, neuron.iloc[17573:17598]]) # Chandelier
#df = pd.concat([df, neuron.iloc[17599:17648]]) # Double bouquet
#df = pd.concat([df, neuron.iloc[17649:17785]]) # Martinotti
#df = pd.concat([df, neuron.iloc[17786:19829]]) # Nitrergic

#df = pd.concat([df, neuron.iloc[19830:21436]]) # Astrocytes
#df = pd.concat([df, neuron.iloc[21437:27882]]) # Microglia

# Drop row having at least 1 missing value
df = df.dropna()

# Creating an instance of Labelencoder
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
# Assigning numerical value and storing it
df[["Target"]] = df[["Target"]].apply(enc.fit_transform)

# Divide the data, y the variable to predict (Target) and X the features
X = df[df.columns[1:]]
y = df['Target']

# Splitting the data : training and test (20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Scaling the data
from sklearn.preprocessing import QuantileTransformer
Normalize = QuantileTransformer(n_quantiles=1000, output_distribution="uniform")
# Transform data
X_train = Normalize.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = X.columns)
X_test = Normalize.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns = X.columns)


# Embedded decision tree classifier

def embedded_decision_tree_classifier(X, y, k_features, output_folder=None):
    '''
    Here we use decision tree classifier to select features. We select the k best features (k_features)
    
    Inputs:
        - X (features) DataFrame
        - y (target) DataFrame
    
    '''
    
    print("\n")
    print("Decision Tree Regressor Features Importance: started")
    print("\n")

    # define the model
    model = DecisionTreeClassifier()
    # fit the model
    model.fit(X, y)
    # get importance
    importance = model.feature_importances_
    # Get features name
    feature_names = [f"{i}" for i in X.columns]

    # create a data frame to visualize features importance
    features_importance = pd.DataFrame({"Features": feature_names, "Importances":importance})
    features_importance.set_index('Importances')

    # Print features importance
    print("\n")
    print("Features Importances:")
    print("\n")
    print(features_importance)
    if output_folder is not None:
        features_importance.to_csv(output_folder+'Decision_Tree_Classifier_Features_Importance.csv', index=False)

    if output_folder is not None:
        # plot feature importance
        features_importance.plot(kind='bar',x='Features',y='Importances')
        pyplot.title('Decision Tree Classifier Features Importance')
        pyplot.tight_layout()
        pyplot.savefig(output_folder+'Decision_Tree_Classfier_Features_Importance.png')
    
    # Select the k most important features
    features_columns = []
    # Order the features importance dataframe
    df = pd.DataFrame(data = features_importance.sort_values(by='Importances', key=abs,ascending=False))
    # Put the k most important features in features_columns
    for x in range(k_features):
        features_columns = features_columns + [df.iloc[x][0]]

    # Create a new DataFrame with selected features
    df_data = pd.DataFrame(data = X, columns = features_columns)
    
    print("\n")
    print("Decision Tree Classifier Features Importance: DataFrame")
    print("\n")
    print(df_data)
    
    return df_data

# Feature selection (we we take the top 10 features)
k_features = 10
X_train = embedded_decision_tree_classifier(X_train, y_train, k_features)
X_test = pd.DataFrame(data = X_test, columns = X_train.columns)

from program_source.qka import FeatureMap

d = 10  # feature dimension is twice the qubit number (To check)

em = [[0,1],[2,3],[4,5],[6,7],[8,9],[1,2],[3,4],[5,6],[7,8]]  # we'll match this to the 10-qubit graph

fm = FeatureMap(feature_dimension=d, entangler_map=em)  # define the feature map
initial_point = [0.1]  # set the initial parameter for the feature map


from qiskit.tools.visualization import circuit_drawer

circuit_drawer(
    fm.construct_circuit(x=X_train[0], parameters=initial_point),
    output="text",
    fold=200,
)

C = 1  # SVM soft-margin penalty
maxiters = 10  # number of SPSA iterations

initial_layout = [9, 8, 11, 14, 16, 19, 22, 25, 24, 23]       # see figure above for the 10-qubit graph

print(service.program("quantum-kernel-alignment"))

def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}\n")

    program_inputs = {
    "feature_map": fm,
    "data": X_train,
    "labels": y_train,
    "initial_kernel_parameters": initial_point,
    "maxiters": maxiters,
    "C": C,
    "initial_layout": initial_layout,
}

options = {"backend_name": backend.name}

job = service.run(
    program_id="quantum-kernel-alignment",
    options=options,
    inputs=program_inputs,
    callback=interim_result_callback,
)

print(job.job_id)
result = job.result()

ImportError: cannot import name 'SymbolicPulse' from 'qiskit.pulse.library' (/usr/local/lib/python3.9/site-packages/qiskit/pulse/library/__init__.py)

In [ ]:
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")

from matplotlib import pyplot as plt
from pylab import cm

plt.rcParams["font.size"] = 20
plt.imshow(
    result["aligned_kernel_matrix"] - np.identity(2 * num_train),
    cmap=cm.get_cmap("bwr", 20),
)
plt.show()